In [1]:
import numpy as np
from keras.layers import Dense, LSTM, Dropout, Convolution2D, MaxPool2D, Flatten, Reshape
from keras.models import Sequential
from keras.utils import to_categorical
from seqframe.seqframe import seq_frame
from keras.regularizers import L2


2023-07-09 04:03:36.445149: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 04:03:36.446277: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-09 04:03:36.474664: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-09 04:03:36.475190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 04:03:37.077769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
no_sequences = 30
no_frames = 1500
words = ['hello', 'world', 'i_am', 'yash']
PATH = '/home/yash/Desktop/Code/trackpad_text_detection/word_data/'

# Augmentation

In [3]:
from scipy.ndimage import rotate
import cv2

def random_resize_crop(image, output_shape):
    if image is None:
        raise ValueError("Invalid input image.")

    # Randomly select a scaling factor between 0.8 and 1.2
    scale_factor = np.random.uniform(0.7, 1.4)

    # Calculate the maximum possible size for the crop
    max_crop_height = int(output_shape[0] / scale_factor)
    max_crop_width = int(output_shape[1] / scale_factor)

    # Calculate the upper bounds for random crop position
    max_start_h = max(image.shape[0] - max_crop_height, 0)
    max_start_w = max(image.shape[1] - max_crop_width, 0)

    # Randomly select the crop position
    start_h = np.random.randint(0, max_start_h + 1)
    start_w = np.random.randint(0, max_start_w + 1)

    # Calculate the end position of the crop
    end_h = start_h + max_crop_height
    end_w = start_w + max_crop_width

    # Perform the crop
    cropped_image = image[start_h:end_h, start_w:end_w]

    # Resize the cropped image to the desired output shape using OpenCV
    resized_image = cv2.resize(cropped_image, output_shape[::-1])

    return resized_image



def random_rotate(image):
    # Randomly select an angle between -15 and 15 degrees
    angle = np.random.uniform(-15, 15)

    # Get the image center coordinates
    image_height, image_width = image.shape[:2]
    center_x = image_width // 2
    center_y = image_height // 2

    # Define the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)

    # Apply the rotation to the image using OpenCV
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image_width, image_height), borderMode=cv2.BORDER_REPLICATE)

    return rotated_image


def augment(img):
    temp = random_resize_crop(img, img.shape)
    temp = random_rotate(temp)
    return temp 

In [4]:
# import matplotlib.pyplot as plt

# a = np.load('word_data/hello_/0/1099.npy')
# b = augment(a)
# plt.imshow(b.T)

# assert b.shape==a.shape


# Get Samples

In [5]:
m, n  = (1919, 1079)
scale = 10
sample_rate = 5
import os
samples = []
for i, word in enumerate(words):
    for sequence in range(no_sequences):
        frames = []
        x = np.load(os.path.join(PATH, word, f'{sequence}.npy'))
        temp = np.zeros((m//scale, n//scale))
        for j, coord in enumerate(x):
            temp[int(coord[0]//scale)][int(coord[1]//scale)] = 1
            temp = temp.astype('uint8')
            if j%sample_rate == 0:
                aug_temp = augment(temp)
                frames.append(aug_temp)
        samples.append(np.array(frames))
x = np.array(samples)


In [6]:
x.shape
x = x.reshape((-1, 191, 107, 1))

In [7]:
x.shape

(36000, 191, 107, 1)

In [8]:
y = np.array([i for i, _ in enumerate(words) for j in range(no_sequences*int(np.ceil(no_frames/sample_rate)))])
y = to_categorical(y)

In [9]:
y.shape
# 13200/110

(36000, 4)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.1, shuffle=True, random_state=12)

In [11]:

model = Sequential()
model.add(Convolution2D(32, kernel_size=(5,5), strides=3))
model.add(MaxPool2D(pool_size=(2,2), strides=2))

model.add(Convolution2D(64, kernel_size=(3,3), strides=2))
model.add(MaxPool2D(pool_size=(2,2), strides=2))

model.add(Convolution2D(128, kernel_size=(3,3), strides=1))
model.add(MaxPool2D(pool_size=(2,2), strides=2))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# # model.add(Reshape((-1, 256,1)))

# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(Dropout(0.1))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(Dropout(0.1))

# model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu', kernel_regularizer=L2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu', kernel_regularizer=L2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu', kernel_regularizer=L2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [12]:
# _, _, m, n = x.shape
model.build(X_train.shape)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (32400, 63, 35, 32)       832       
                                                                 
 max_pooling2d (MaxPooling2  (32400, 31, 17, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32400, 15, 8, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32400, 7, 4, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (32400, 5, 2, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (32400, 2, 1, 128)        0

# CV 

In [13]:
# from sklearn.model_selection import StratifiedKFold
# from tqdm import tqdm
# state_accuracy=[]
# for i in tqdm(range(3)):
#     oof_scores = []
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=i)
#     for train_index, valid_index in skf.split(X_train, np.argmax(y_train,axis=1)):
#         xtrain, ytrain = X_train[train_index], y_train[train_index]
#         xvalid, yvalid = X_train[valid_index], y_train[valid_index]
        
#         model.fit(xtrain, ytrain, epochs=5)
#         preds=model.predict(xvalid)
#         ytrue = np.argmax(yvalid, axis=1)
#         yhat = np.argmax(preds, axis=1)
#         oof_scores.append(accuracy_score(ytrue, yhat))
#     state_accuracy.append(np.array(oof_scores))

In [14]:
# np.mean(np.mean(state_accuracy, axis=1))

In [15]:

model.fit(X_train,y_train, epochs=5)

In [16]:
preds = model.predict(X_test)

In [17]:
y_true = np.argmax(y_test, axis=1)
y_hat = np.argmax(preds, axis=1)

In [18]:
accuracy_score(y_true, y_hat)

In [19]:
# model.save('model2_1.h5')

In [20]:
np.argmax(np.mean(preds, axis=1))

In [22]:
# model.fit(x,y, epochs=5)
# model.save('model2_2.h5')

Epoch 1/5
1125/1125 [==============================] - 10s 9ms/step - loss: 1.1092 - categorical_accuracy: 0.5861
Epoch 2/5
1125/1125 [==============================] - 10s 9ms/step - loss: 0.6053 - categorical_accuracy: 0.7802
Epoch 3/5
1125/1125 [==============================] - 10s 9ms/step - loss: 0.5026 - categorical_accuracy: 0.8177
Epoch 4/5
1125/1125 [==============================] - 10s 9ms/step - loss: 0.4563 - categorical_accuracy: 0.8357
Epoch 5/5
1125/1125 [==============================] - 10s 9ms/step - loss: 0.4198 - categorical_accuracy: 0.8505


/home/yash/Desktop/Code/trackpad_text_detection/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
